In [22]:
import geopandas as gpd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='pacaas-novas')

In [23]:
pacaas_area = gpd.read_file('C:/Users/ppgar/Downloads/QGIS/Pacaas_novos/pacaas_novas.geojson')

In [24]:
pacaas_area = pacaas_area.geometry.iloc[0].__geo_interface__

In [25]:
pacaas_area = ee.Geometry(pacaas_area)
buffer = pacaas_area.buffer(5000)
buffer_diff = buffer.difference(pacaas_area)

In [26]:
def mask_clouds(image):
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)

satelite_2016 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-01-01', '2016-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])
satelite_2016_buffer = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-01-01', '2016-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer_diff).select(['B11', 'B8', 'B4'])
satelite_2024 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2024-01-01', '2024-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])
satelite_2024_buffer = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2024-01-01', '2024-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer_diff).select(['B11', 'B8', 'B4'])

In [27]:
def calc_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [28]:
def calc_evi(image):
    nir = image.select('B8')
    red = image.select('B4')
    blue = image.select('B2')
    evi = nir.subtract(red).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).multiply(2.5).rename('EVI')
    return image.addBands(evi)

In [29]:
images = [satelite_2016, satelite_2016_buffer, satelite_2024, satelite_2024_buffer]
[satelite_2016_ndvi, satelite_2016_buffer_ndvi, satelite_2024_ndvi, satelite_2024_buffer_ndvi] = [calc_ndvi(i) for i in images]
[mean_ndvi_2016, mean_ndvi_buffer_2016, mean_ndvi_2024, mean_ndvi_buffer_2024] = [x.reduceRegion(reducer = 'median') for x in [satelite_2016_ndvi, satelite_2016_buffer_ndvi, satelite_2024_ndvi, satelite_2024_buffer_ndvi]]


In [35]:
m = geemap.Map()
m.addLayer(satelite_2024_buffer_ndvi)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…